In [ ]:
# Import libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv('../input/machine-learning-24-hrs-hackathon/train_SJC.csv')
# import data

# ETL

In [ ]:
df.info()
# Columns need to be named correctly

## Renaming Columns

In [ ]:
df.columns = df.iloc[0]
# Rename all the columns with first row

In [ ]:
df.head()

In [ ]:
column_indices = [2,7,11]
new_names = ['DateReported','DependentsOther','DaysWorkedPerWeek']
old_names = df.columns[column_indices]
df.rename(columns=dict(zip(old_names, new_names)), inplace=True)
# Rename three columns whose column names were nan

In [ ]:
df.head()

In [ ]:
df = df.iloc[1:]
# Delete first row

## Removing Duplicate rows

In [ ]:
df.drop_duplicates(inplace=True)
# No duplicate rows in this case

In [ ]:
df.info()
# data has missing values and data types are not appropriate

## Changing data type

In [ ]:
df[['Age', 'DependentChildren','WeeklyWages','HoursWorkedPerWeek','InitialIncurredCalimsCost','UltimateIncurredClaimCost']] = df[['Age', 'DependentChildren','WeeklyWages','HoursWorkedPerWeek','InitialIncurredCalimsCost','UltimateIncurredClaimCost']].astype(float)

# While downloading dataset, due to presence of first row many column data types are not correct

In [ ]:
df["DateTimeOfAccident"]=pd.to_datetime(df["DateTimeOfAccident"])
df["DateReported"]=pd.to_datetime(df["DateReported"])

In [ ]:
df.dtypes

In [ ]:
df.loc[:,['DateTimeOfAccident','DateReported']]

In [ ]:
# define functions

# Create Data audit Report for continuous variables
def continuous_var_summary(x):
    return pd.Series([x.count(), x.isnull().sum(), x.sum(), x.mean(), x.median(),  
                      x.std(), x.var(), x.min(), x.quantile(0.01), x.quantile(0.05),
                          x.quantile(0.10),x.quantile(0.25),x.quantile(0.50),x.quantile(0.75), 
                              x.quantile(0.90),x.quantile(0.95), x.quantile(0.98), x.quantile(0.99).round(2),x.max()], 
                  index = ['N', 'NMISS', 'SUM', 'MEAN','MEDIAN', 'STD', 'VAR', 'MIN', 'P1', 
                            'P5' ,'P10' ,'P25' ,'P50' ,'P75' ,'P90' ,'P95' ,'P98','P99' ,'MAX'])

# Create Data audit Report for categorical variables
def categorical_var_summary(x):
    Mode = x.value_counts().sort_values(ascending = False)[0:1].reset_index()
    return pd.Series([x.count(), x.isnull().sum(), Mode.iloc[0, 0], Mode.iloc[0, 1], 
                          round(Mode.iloc[0, 1] * 100/x.count(), 2)], 
                  index = ['N', 'NMISS', 'MODE', 'FREQ', 'PERCENT'])
    

# An utility function to create dummy variable
def create_dummies(df, colname):
    col_dummies = pd.get_dummies(df[colname], prefix = colname, drop_first = True)
    df = pd.concat([df, col_dummies], axis = 1)
    df.drop(colname, axis = 1, inplace = True )
    return df

# Missing value imputation for categorical and continuous variables
def missing_imputation(x, stats = 'mean'):
    if (x.dtypes == 'float64'):   # | (x.dtypes == 'int64')
        x = x.fillna(x.mean()) #  if stats == 'mean' else x.fillna(x.median()
    else:
        x = x.fillna(x.mode())
    return x

# Data Visualization

## Outlier treatment

In [ ]:
# Continuous features 
df.describe().T
# We aren't able to observe change in last few percentiles, so we will create a function to get detailed analysis

In [ ]:
df[df['HoursWorkedPerWeek']>500].shape

In [ ]:
sns.boxplot(df["HoursWorkedPerWeek"])

In [ ]:
df[df['WeeklyWages']>5000.0].shape
# Since there are 1800 rows which shows that we will not be able to predict high claim cost if we do outlier treatment, so we can 

In [ ]:
sns.boxplot(df["WeeklyWages"])

In [ ]:
df[df['UltimateIncurredClaimCost']>1373660.7]


In [ ]:
sns.boxplot(df["UltimateIncurredClaimCost"])
# Since one data point gives a very high value we will treat it as outlier and remove it from data

In [ ]:
df = df[df.ClaimNumber != 'WC9901999']

In [ ]:
df.drop(['ClaimNumber'],axis=1,inplace=True)
# since it is unique, this can be dropped

In [ ]:
df_conti = df.loc[:, (df.dtypes == 'float64') | (df.dtypes == 'int64')]
df_cat = df.loc[:,(df.dtypes == 'object')|(df.dtypes=="category")]

In [ ]:
df_conti.apply(continuous_var_summary).T.round(1) 

In [ ]:
df_conti= df_conti.apply(lambda x: x.clip(lower = x.dropna().quantile(0.01), upper = x.quantile(0.99)))
df_conti.apply(continuous_var_summary).T.round(1)
# Outlier treatment at 99th percentile

## Missing value treatment

In [ ]:
df.isnull().sum().plot(kind='bar')
plt.show()

In [ ]:
df.columns[df.isna().any()]

In [ ]:
# df['WeeklyWages'] = df['WeeklyWages'].fillna((df['WeeklyWages'].mean()))
# df['HoursWorkedPerWeek'] = df['HoursWorkedPerWeek'].fillna((df['HoursWorkedPerWeek'].mean()))
# df['MaritalStatus'] = df['MaritalStatus'].fillna((df['MaritalStatus'].mode()[0]))

# Individual treatment of columns may create problem in pre-processing of test data, so we will try to use something more general

In [ ]:
df_cat['MaritalStatus'] = df_cat['MaritalStatus'].fillna('U')
# Because of presence of a category U, we will replace nan with U

In [ ]:
from statistics import mode
df_conti= df_conti.apply(missing_imputation)
df_cat = df_cat.apply(missing_imputation)

In [ ]:
df.info()
# No missing values

In [ ]:
# Categorical features 
df.describe(include=['object']).T

In [ ]:
df_new = pd.concat([df_conti, df_cat,df.loc[:,['DateTimeOfAccident','DateReported']]], axis=1)

In [ ]:
df_new.head()

In [ ]:
df_new.columns

In [ ]:
df_processed=df_new.copy()

In [ ]:
# df_new=df_processed.copy()

## Model made by dropping columns

In [ ]:
df_new.drop(['ClaimDescription','DateTimeOfAccident', 'DateReported'], axis=1, inplace=True)

In [ ]:
df_new.head()

# data for modelling

In [ ]:
target_train=df_new['UltimateIncurredClaimCost']
target_train

In [ ]:
x_df=df_new.loc[:, df_new.columns != 'UltimateIncurredClaimCost']

In [ ]:
import sklearn.preprocessing as pre
import sklearn.model_selection as ms
import sklearn.linear_model as lm

In [ ]:
le=pre.LabelEncoder()
for x in x_df.select_dtypes(include='object').columns.tolist():
    x_df[x]=le.fit_transform(x_df[x])

In [ ]:
X_scale=pre.minmax_scale(x_df)
Y=target_train

In [ ]:
x_train,x_test,y_train,y_test=ms.train_test_split(X_scale,Y,test_size=0.3,random_state=1234457)

In [ ]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

## Linear regression model

In [ ]:
glm=lm.LinearRegression()

In [ ]:
glm.fit(x_train,y_train)

In [ ]:
glm.score(x_test,y_test)

In [ ]:
glm.score(x_train,y_train)

In [ ]:
import sklearn.metrics as mt
a=mt.mean_squared_error(y_pred=glm.predict(x_test),y_true=y_test)
import math
math.sqrt(a)

# Other models that I tried but haven't been used for final prediction

## KNN

In [ ]:
import sklearn.neighbors as NN

In [ ]:
KNN=NN.KNeighborsRegressor(n_neighbors=7)

In [ ]:
KNN.fit(x_train,y_train)

In [ ]:
KNN.predict(x_test)

In [ ]:
import sklearn.metrics as mt

import math

a=mt.mean_squared_error(y_pred=KNN.predict(x_test),y_true=y_test)
math.sqrt(a)

## Random Forest

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 500 decision trees
rf = RandomForestRegressor(n_estimators = 500, random_state = 42351, max_depth=5)
# Train the model on training data
rf.fit(x_train,y_train)
# ,max_leaf_nodes=7,min_samples_split=50

In [ ]:
# Use the forest's predict method on the test data
predictions = rf.predict(x_test)

In [ ]:
a=mt.mean_squared_error(y_pred=rf.predict(x_test),y_true=y_test)
math.sqrt(a)

In [ ]:
rf.score(x_train,y_train)

In [ ]:
rf.score(x_test,y_test)

# Date-time column Transformations

In [ ]:
df_1=df_processed.copy()

In [ ]:
df_1['YearOfAccident']  = pd.DatetimeIndex(df_1['DateTimeOfAccident']).year
df_1['MonthOfAccident']  = pd.DatetimeIndex(df_1['DateTimeOfAccident']).month
df_1['DayOfAccident']  = pd.DatetimeIndex(df_1['DateTimeOfAccident']).day
df_1['WeekdayOfAccident']  = pd.DatetimeIndex(df_1['DateTimeOfAccident']).day_name()
df_1['HourOfAccident']  = pd.DatetimeIndex(df_1['DateTimeOfAccident']).hour
df_1['YearReported']  = pd.DatetimeIndex(df_1['DateReported']).year
df_1['DaysDelayed'] = (pd.DatetimeIndex(df_1['DateReported']).date - pd.DatetimeIndex(df_1['DateTimeOfAccident']).date)

In [ ]:
df_1.dtypes

In [ ]:
df_1['DaysDelayed']=df_1['DaysDelayed'].dt.days

In [ ]:
df_1.head()

In [ ]:
df_1.drop(['ClaimDescription','DateTimeOfAccident', 'DateReported'], axis=1, inplace=True)

In [ ]:
df_1.head()

# data for modelling

In [ ]:
target_train=df_1['UltimateIncurredClaimCost']
target_train

In [ ]:
x_df=df_1.loc[:, df_1.columns != 'UltimateIncurredClaimCost']

In [ ]:
le=pre.LabelEncoder()
for x in x_df.select_dtypes(include='object').columns.tolist():
    x_df[x]=le.fit_transform(x_df[x])

In [ ]:
x_df.head()

In [ ]:
X_scale=pre.minmax_scale(x_df)
Y=target_train

In [ ]:
x_train,x_test,y_train,y_test=ms.train_test_split(x_df,Y,test_size=0.3,random_state=1234457)

In [ ]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

## Linear regression model

In [ ]:
import sklearn.preprocessing as pre
import sklearn.model_selection as ms
import sklearn.linear_model as lm

In [ ]:
glm_=lm.LinearRegression()

In [ ]:
glm_.fit(x_train,y_train)

In [ ]:
a=mt.mean_squared_error(y_pred=glm_.predict(x_test),y_true=y_test)
math.sqrt(a)

In [ ]:
glm_.score(x_test,y_test)


In [ ]:
glm_.score(x_train,y_train)

## Random Forest

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf_ = RandomForestRegressor(n_estimators = 500, random_state = 42351, max_depth=5)
# Train the model on training data
rf_.fit(x_train,y_train);

In [ ]:
# Use the forest's predict method on the test data
predictions = rf_.predict(x_test)

In [ ]:
a=mt.mean_squared_error(y_pred=rf_.predict(x_test),y_true=y_test)
math.sqrt(a)

In [ ]:
rf_.score(x_train,y_train)

In [ ]:
rf_.score(x_test,y_test)

# Creating columns from Date-time columns

In [ ]:
df_11=df_processed.copy()

In [ ]:
df_11.columns

In [ ]:
df_11['HoursWorkedPerWeek']=pd.cut(df_11['HoursWorkedPerWeek'],bins=[1,20,35,40,80],labels=['less','mid','okay','high'])
df_11['DaysWorkedPerWeek']=pd.cut(df_11['DaysWorkedPerWeek'],bins=[1,4,5,7],labels=['less','okay','high'])
df_11['Age']=pd.cut(df_11['Age'],bins=[1,20,35,40,80],labels=['less','mid','okay','high'])
# transformation will help in better prediction

In [ ]:
df_11['YearOfAccident']  = pd.DatetimeIndex(df_11['DateTimeOfAccident']).year
df_11['MonthOfAccident']  = pd.DatetimeIndex(df_11['DateTimeOfAccident']).month
df_11['DayOfAccident']  = pd.DatetimeIndex(df_11['DateTimeOfAccident']).day
df_11['WeekdayOfAccident']  = pd.DatetimeIndex(df_11['DateTimeOfAccident']).day_name()
df_11['HourOfAccident']  = pd.DatetimeIndex(df_11['DateTimeOfAccident']).hour
df_11['YearReported']  = pd.DatetimeIndex(df_11['DateReported']).year
df_11['DaysDelayed'] = (pd.DatetimeIndex(df_11['DateReported']).date - pd.DatetimeIndex(df_11['DateTimeOfAccident']).date)

In [ ]:
df_11['DaysDelayed']=df_11['DaysDelayed'].dt.days

In [ ]:
df_11.dtypes

In [ ]:
df_11.corr()

In [ ]:
# high correlation 'YearReported','YearOfAccident'

In [ ]:
df_11.drop(['DateTimeOfAccident','DateReported','ClaimDescription','YearOfAccident'],axis=1,inplace=True)

In [ ]:
df_11['MonthOfAccident']=df_11['MonthOfAccident'].astype('str')
df_11['WeekdayOfAccident']=df_11['WeekdayOfAccident'].astype('str')
df_11['DependentChildren']=df_11['DependentChildren'].astype('str')
df_11['DependentsOther']=df_11['DependentsOther'].astype('str')

In [ ]:
df_11.columns

In [ ]:
df_conti = df_11.loc[:, (df_11.dtypes == 'float64') | (df_11.dtypes == 'int64')]
df_cat = df_11.loc[:,(df_11.dtypes == 'object')|(df_11.dtypes=="category")]

In [ ]:
df_conti.columns

### Creating dummy variables

In [ ]:
for c_feature in df_cat:
    df_cat[c_feature] = df_cat[c_feature].astype('category')
    df_cat = create_dummies(df_cat,c_feature)

In [ ]:
df_cat.columns

In [ ]:
df_11 = pd.concat([df_conti, df_cat], axis=1)

In [ ]:
df_11.dtypes

In [ ]:
df_11.head()

# data for modelling

In [ ]:
target_train=df_11['UltimateIncurredClaimCost']
target_train

In [ ]:
x_df=df_11.loc[:, df_11.columns != 'UltimateIncurredClaimCost']

In [ ]:
# le=pre.LabelEncoder()
# for x in x_df.select_dtypes(include='object').columns.tolist():
#     x_df[x]=le.fit_transform(x_df[x])

In [ ]:
x_df.head()

In [ ]:
# X_scale=pre.minmax_scale(x_df)
Y=target_train

In [ ]:
x_train,x_test,y_train,y_test=ms.train_test_split(x_df,Y,test_size=0.3,random_state=1234457)

In [ ]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

## Linear regression model

In [ ]:
import sklearn.preprocessing as pre
import sklearn.model_selection as ms
import sklearn.linear_model as lm

In [ ]:
glm__=lm.LinearRegression()

In [ ]:
glm__.fit(x_train,y_train)

In [ ]:
glm__.score(x_test,y_test)


In [ ]:
glm__.score(x_train,y_train)

## Random Forest

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf__ = RandomForestRegressor(n_estimators = 500, random_state = 42351, max_depth=15,max_leaf_nodes=7,min_samples_split=50)
# Train the model on training data
rf__.fit(x_train,y_train);

In [ ]:
import sklearn.metrics as mt
a=mt.mean_squared_error(y_pred=rf__.predict(x_test),y_true=y_test)
import math
math.sqrt(a)

In [ ]:
rf__.score(x_train,y_train)

In [ ]:
rf__.score(x_test,y_test)

## Testing dataset

In [ ]:
df_test=pd.read_csv('../input/machine-learning-24-hrs-hackathon/Test_SJC.csv')

In [ ]:
# Changing data type
df_test[['Age', 'DependentChildren','WeeklyWages','HoursWorkedPerWeek','InitialIncurredCalimsCost']] = df_test[['Age', 'DependentChildren','WeeklyWages','HoursWorkedPerWeek','InitialIncurredCalimsCost']].astype(float)
df_test["DateTimeOfAccident"]=pd.to_datetime(df_test["DateTimeOfAccident"])
df_test["DateReported"]=pd.to_datetime(df_test["DateReported"])
df_test.drop(['ClaimNumber'],axis=1,inplace=True)

# Outlier treatment
df_test_conti = df_test.loc[:, (df_test.dtypes == 'float64') | (df_test.dtypes == 'int64')]
df_test_cat = df_test.loc[:,(df_test.dtypes == 'object')|(df_test.dtypes=="category")]

df_test_conti= df_test_conti.apply(lambda x: x.clip(lower = x.dropna().quantile(0.01), upper = x.quantile(0.99)))
# df_test_conti.apply(continuous_var_summary).T.round(1)
# Outlier treatment at 99th percentile

# Missing value treatment
df_test_cat['MaritalStatus'] = df_test_cat['MaritalStatus'].fillna('U')

df_test_conti= df_test_conti.apply(missing_imputation)
df_test_cat = df_test_cat.apply(missing_imputation)

df_new_test = pd.concat([df_test_conti, df_test_cat,df_test.loc[:,['DateTimeOfAccident','DateReported']]], axis=1)

# Removing some variables
df_new_test.drop(['ClaimDescription','DateTimeOfAccident', 'DateReported'], axis=1, inplace=True)


# Label-encoding for categorical variables
# le=pre.LabelEncoder()
for x in df_new_test.select_dtypes(include='object').columns.tolist():
    df_new_test[x]=le.fit_transform(df_new_test[x])
    
X_scale_test=pre.minmax_scale(df_new_test)

# This model has been used for prediction

In [ ]:
predictions = glm.predict(X_scale_test)
csv = pd.read_csv("../input/machine-learning-24-hrs-hackathon/sample_submission.csv")
csv["UltimateIncurredClaimCost"]=predictions
csv.to_csv("20BDA68.csv", index = False)

### Here, I have tried to use both linear regression and random forest model on different sets of pre-processed data, but I finalized my model based on simplicity to explain model to client. And even though we increase complexity results doesn't differ much. So, I have chosen the least complex model.
